In [ ]:
import os
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from tqdm import tqdm
from torch import optim
import logging
import pennylane as qml
import numpy as np
import random
import time

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

qubit_range = range(2, 25)  
layers = 3
times = []

for qubits in qubit_range:

    config = {
        "qubits": qubits,
        "layers": layers,
    }

    w = np.random.rand(qubits * 2)

    dev = qml.device('lightning.kokkos', wires=qubits)

    @qml.qnode(dev, interface='torch', diff_method='adjoint')
    def gen_circuit(w):
        z1 = random.uniform(-1, 1)
        z2 = random.uniform(-1, 1)
        for i in range(qubits):
            qml.RY(np.arcsin(z1), wires=i)
            qml.RZ(np.arcsin(z2), wires=i)
        for l in range(layers):
            for i in range(qubits):
                qml.RY(w[i], wires=i)
            for i in range(qubits - 1):
                qml.CNOT(wires=[i, i + 1])
                qml.RZ(w[i + qubits], wires=i + 1)
                qml.CNOT(wires=[i, i + 1])
        return [qml.expval(qml.PauliZ(i)) for i in range(qubits)]

    x = torch.randn(1, 3, 64, 64, device=device)
    Ɛ = torch.zeros_like(x, device=device)

    start_time = time.time()

    for i in range(x.shape[1]): 
        for j in range(x.shape[2]):
            values = []
            for k in range(0, x.shape[3]):
                if k % config["qubits"] == 0:  
                    values = gen_circuit(w)
                    random.shuffle(values)
                Ɛ[:, i, j, k] = values[k % config["qubits"]]

    end_time = time.time()

    elapsed_time = end_time - start_time
    times.append(elapsed_time)
    print(f"Qubits: {qubits}, Execution Time: {elapsed_time:.2f} seconds")

plt.figure(figsize=(10, 5))
plt.plot(qubit_range, times, marker='o')
plt.title('Execution Time for Image Processing with Varying Qubits')
plt.xlabel('Number of Qubits')
plt.ylabel('Execution Time (seconds)')
plt.grid(True)
plt.show()
